In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
import math

# Définir une fonction pour calculer le RMSE
def rmse_scorer(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

# Créer un scorer personnalisé
from sklearn.metrics import make_scorer, mean_squared_error
rmse_scorer_sklearn = make_scorer(rmse_scorer, greater_is_better=False)

# Définir les paramètres à tester (version légère pour réduire le temps de calcul)
param_grid = {
    'base_estimator__n_estimators': [100, 200],
    'base_estimator__learning_rate': [0.1, 0.2],
    'base_estimator__max_depth': [3, 5]
}

# Configurer la validation croisée
cv = KFold(n_splits=5, shuffle=True, random_state=42)

In [2]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import RegressorChain

# Créer le modèle de base
base_model = GradientBoostingRegressor(random_state=42)
chain_model = RegressorChain(base_model, order=[0, 1, 2, 3, 4], random_state=42)

# Configurer GridSearchCV
grid_search = GridSearchCV(
    estimator=chain_model,
    param_grid=param_grid,
    scoring=rmse_scorer_sklearn,
    cv=cv,
    verbose=1,
    n_jobs=-1  # Utiliser tous les cœurs disponibles
)

In [4]:
# Charger les données nécessaires
import polars as pl
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Charger le dataset silver
silver_df = pl.read_csv('../data/silver_mega_evolutions.csv')

# Définir les colonnes à prédire
target_cols = [
    'evolved_attack', 'evolved_defense', 
    'evolved_sp_attack', 'evolved_sp_defense', 'evolved_speed'
]

# Colonnes d'entrée
input_cols = [col for col in silver_df.columns if col.startswith('base_')]

# Préparer X et y
X = silver_df.select(input_cols)
y = silver_df.select(target_cols)

# Convertir en numpy
X_np = X.to_numpy()
y_np = y.to_numpy()

# Diviser en train/test
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

# Standardiser
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train_np)
y_train_scaled = scaler_y.fit_transform(y_train_np)
X_test_scaled = scaler_X.transform(X_test_np)
y_test_scaled = scaler_y.transform(y_test_np)

In [5]:
import mlflow

# Configuration de l'expérience MLflow
mlflow.set_experiment("mega_evolution_optimization")

# Activer l'autologging pour scikit-learn
mlflow.sklearn.autolog(log_models=True, log_input_examples=True)

2025/05/21 22:52:26 INFO mlflow.tracking.fluent: Experiment with name 'mega_evolution_optimization' does not exist. Creating a new experiment.


In [6]:
# Exécuter la recherche par grille avec tracking MLflow
with mlflow.start_run(run_name="GB_Chain_GridSearch"):
    print("Démarrage de la recherche d'hyperparamètres pour Gradient Boosting Chain...")
    
    # Log des paramètres de la recherche
    mlflow.log_param("cv_folds", cv.n_splits)
    mlflow.log_param("param_grid", str(param_grid))
    
    # Exécuter la recherche
    grid_search.fit(X_train_scaled, y_train_scaled)
    
    # Afficher les meilleurs hyperparamètres
    print("\nMeilleurs hyperparamètres trouvés:")
    print(grid_search.best_params_)
    print(f"Meilleur score RMSE (validation croisée): {-grid_search.best_score_:.4f}")
    
    # Log des meilleurs paramètres
    for param, value in grid_search.best_params_.items():
        mlflow.log_param(f"best_{param}", value)
    
    # Log du meilleur score
    mlflow.log_metric("best_cv_rmse", -grid_search.best_score_)

Démarrage de la recherche d'hyperparamètres pour Gradient Boosting Chain...
Fitting 5 folds for each of 8 candidates, totalling 40 fits


2025/05/21 22:54:28 INFO mlflow.sklearn.utils: Logging the 5 best runs, 3 runs will be omitted.



Meilleurs hyperparamètres trouvés:
{'base_estimator__learning_rate': 0.2, 'base_estimator__max_depth': 3, 'base_estimator__n_estimators': 100}
Meilleur score RMSE (validation croisée): 0.7512


In [9]:
import polars as pl

# Exécuter la recherche par grille avec tracking détaillé MLflow
with mlflow.start_run(run_name="GB_Chain_GridSearch_Main"):
    print("Démarrage de la recherche d'hyperparamètres pour Gradient Boosting Chain...")
    
    # Log des paramètres de la recherche
    mlflow.log_param("cv_folds", cv.n_splits)
    mlflow.log_param("param_grid", str(param_grid))
    
    # Exécuter la recherche
    grid_search.fit(X_train_scaled, y_train_scaled)
    
    # Afficher les meilleurs hyperparamètres
    print("\nMeilleurs hyperparamètres trouvés:")
    print(grid_search.best_params_)
    print(f"Meilleur score RMSE (validation croisée): {-grid_search.best_score_:.4f}")
    
    # Log des meilleurs paramètres
    for param, value in grid_search.best_params_.items():
        mlflow.log_param(f"best_{param}", value)
    
    # Log du meilleur score
    mlflow.log_metric("best_cv_rmse", -grid_search.best_score_)
    
    # Convertir les résultats de GridSearch en DataFrame Polars
    # Les résultats de GridSearch sont dans un dictionnaire que nous devons d'abord transformer
    # car certains éléments comme les paramètres ne sont pas directement convertibles en Polars
    cv_results_dict = {k: list(v) if isinstance(v, np.ndarray) else v 
                      for k, v in grid_search.cv_results_.items()}
    
    # Pour les objets de type non standard, convertir en string
    for k, v in cv_results_dict.items():
        if k == 'params':
            cv_results_dict[k] = [str(param_dict) for param_dict in v]
    
    cv_results = pl.DataFrame(cv_results_dict)
    
    # Filtrer et trier les colonnes pertinentes
    filtered_results = cv_results.select(
        ["params", "mean_test_score", "std_test_score", "rank_test_score"]
    ).sort("rank_test_score")
    
    # Afficher les résultats
    print("\nRésultats des différentes combinaisons d'hyperparamètres:")
    print(filtered_results)
    
    # Conversion temporaire en pandas pour MLflow avec le bon format de paramètres
    mlflow.log_table(data=filtered_results.to_pandas(), artifact_file="cv_results.json")
    
    # Log des runs enfants pour chaque combinaison d'hyperparamètres
    for i, row in enumerate(cv_results.iter_rows(named=True)):
        mean_score = row["mean_test_score"]
        std_score = row["std_test_score"]
        rank = row["rank_test_score"]
        
        # Récupérer les paramètres originaux (non convertis en string)
        params = grid_search.cv_results_['params'][i]
        params_str = "_".join([f"{k.split('__')[-1]}={v}" for k, v in params.items()])
        
        # Créer un run enfant pour cette combinaison
        with mlflow.start_run(run_name=f"Params_{params_str}", nested=True):
            # Log des paramètres
            for param_name, param_value in params.items():
                mlflow.log_param(param_name, param_value)
            
            # Log des métriques
            mlflow.log_metric("mean_rmse", -mean_score)
            mlflow.log_metric("std_rmse", std_score)
            mlflow.log_metric("rank", rank)
            
            # Log si c'est le meilleur modèle
            is_best = (rank == 1)
            mlflow.log_param("is_best_model", is_best)

Démarrage de la recherche d'hyperparamètres pour Gradient Boosting Chain...
Fitting 5 folds for each of 8 candidates, totalling 40 fits


2025/05/21 23:09:06 INFO mlflow.sklearn.utils: Logging the 5 best runs, 3 runs will be omitted.



Meilleurs hyperparamètres trouvés:
{'base_estimator__learning_rate': 0.2, 'base_estimator__max_depth': 3, 'base_estimator__n_estimators': 100}
Meilleur score RMSE (validation croisée): 0.7512

Résultats des différentes combinaisons d'hyperparamètres:
shape: (8, 4)
┌─────────────────────────────────┬─────────────────┬────────────────┬─────────────────┐
│ params                          ┆ mean_test_score ┆ std_test_score ┆ rank_test_score │
│ ---                             ┆ ---             ┆ ---            ┆ ---             │
│ str                             ┆ f64             ┆ f64            ┆ i32             │
╞═════════════════════════════════╪═════════════════╪════════════════╪═════════════════╡
│ {'base_estimator__learning_rat… ┆ -0.751199       ┆ 0.164841       ┆ 1               │
│ {'base_estimator__learning_rat… ┆ -0.751204       ┆ 0.164847       ┆ 2               │
│ {'base_estimator__learning_rat… ┆ -0.754194       ┆ 0.163845       ┆ 3               │
│ {'base_estimator__le

In [10]:
from sklearn.metrics import mean_squared_error, r2_score

# Récupérer le meilleur modèle trouvé
best_model = grid_search.best_estimator_

# Faire des prédictions sur l'ensemble de test
y_pred_scaled = best_model.predict(X_test_scaled)

# Reconvertir les prédictions et les vraies valeurs à l'échelle originale
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_test_scaled)

# Évaluer les performances pour chaque stat
print("\nPerformances du modèle optimisé sur l'ensemble de test:")
metrics = {}
rmse_values = []
r2_values = []

for i, col in enumerate(target_cols):
    stat_name = col.replace('evolved_', '')
    rmse = np.sqrt(mean_squared_error(y_true[:, i], y_pred[:, i]))
    r2 = r2_score(y_true[:, i], y_pred[:, i])
    
    rmse_values.append(rmse)
    r2_values.append(r2)
    
    metrics[stat_name] = {"RMSE": rmse, "R²": r2}
    print(f"  {stat_name}: RMSE = {rmse:.2f}, R² = {r2:.2f}")

# Calculer les moyennes
avg_rmse = np.mean(rmse_values)
avg_r2 = np.mean(r2_values)
print(f"  Moyenne: RMSE = {avg_rmse:.2f}, R² = {avg_r2:.2f}")

# Créer un DataFrame Polars pour les résultats
results_df = pl.DataFrame({
    "Stat": list(metrics.keys()),
    "RMSE": [metrics[stat]["RMSE"] for stat in metrics],
    "R²": [metrics[stat]["R²"] for stat in metrics]
})

print("\nRésumé des performances:")
print(results_df)

# Enregistrer ces résultats dans MLflow
with mlflow.start_run(run_name="Final_Model_Evaluation"):
    # Log des hyperparamètres
    for param, value in grid_search.best_params_.items():
        mlflow.log_param(param, value)
    
    # Log des métriques
    for stat_name, values in metrics.items():
        mlflow.log_metric(f"{stat_name}_RMSE", values["RMSE"])
        mlflow.log_metric(f"{stat_name}_R2", values["R²"])
    
    mlflow.log_metric("average_RMSE", avg_rmse)
    mlflow.log_metric("average_R2", avg_r2)
    
    # Log du modèle avec sa signature
    from mlflow.models.signature import infer_signature
    signature = infer_signature(X_test_scaled, y_pred_scaled)
    mlflow.sklearn.log_model(best_model, "final_model", signature=signature)


Performances du modèle optimisé sur l'ensemble de test:
  attack: RMSE = 23.98, R² = 0.57
  defense: RMSE = 24.48, R² = 0.34
  sp_attack: RMSE = 16.35, R² = 0.85
  sp_defense: RMSE = 17.70, R² = 0.10
  speed: RMSE = 13.56, R² = 0.90
  Moyenne: RMSE = 19.21, R² = 0.55

Résumé des performances:
shape: (5, 3)
┌────────────┬───────────┬──────────┐
│ Stat       ┆ RMSE      ┆ R²       │
│ ---        ┆ ---       ┆ ---      │
│ str        ┆ f64       ┆ f64      │
╞════════════╪═══════════╪══════════╡
│ attack     ┆ 23.983158 ┆ 0.56566  │
│ defense    ┆ 24.481485 ┆ 0.33894  │
│ sp_attack  ┆ 16.346381 ┆ 0.850748 │
│ sp_defense ┆ 17.696512 ┆ 0.101748 │
│ speed      ┆ 13.555917 ┆ 0.898879 │
└────────────┴───────────┴──────────┘
